In [1]:
from random import uniform
from math import sqrt
from queue import PriorityQueue

In [2]:
def euclidean_distance(a, b):
    return sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

In [3]:
class EventHandler:
    def __init__(self):
        self.events = PriorityQueue()
        
    def addEvent(self, e):
        self.events.put(e)
    
    def getEvent(self):
        return self.events.get()
    
    def isEmpty(self):
        return self.events.empty()

In [15]:
class Node:
    def __init__(self, maxX, maxY, minS, maxS):
        self.maxX = maxX
        self.maxY = maxY
        self.maxS = maxS
        self.minS = minS
        self.last_waypoint = -1
        self.getNewCoords()
        self.getNewWaypoint()
        self.getNewSpeed()
        
    def getNewCoords(self):
        self.x = uniform(0, self.maxX)
        self.y = uniform(0, self.maxY)
        
    def getNewWaypoint(self):
        self.next_waypoint_x = uniform(0, self.maxX)
        self.next_waypoint_y = uniform(0, self.maxY)
        
    def getNewSpeed(self):
        self.speed = uniform(self.minS, self.maxS)
        
    def getTimeNeeded(self):
        d = euclidean_distance((self.x, self.y), (self.next_waypoint_x, self.next_waypoint_y))
        return d / self.speed
    
    def arrived(self):
        self.x = self.next_waypoint_x
        self.y = self.next_waypoint_y
        self.getNewWaypoint()
        self.getNewSpeed()

In [42]:
class Simulation:
    def __init__(self, N, maxX, maxY, minS, maxS, time_measure):
        self.events = EventHandler()
        self.N = N
        self.maxX = maxX
        self.maxY = maxY
        self.maxS = maxS
        self.minS = minS
        self.time_measure = time_measure
        self.nodes = []
        self.events.addEvent((0, "init", -1))
        
    def init_event_handler(self):
        for n in range(self.N):
            self.nodes.append(Node(self.maxX, self.maxY, self.maxS, self.minS))
            self.events.addEvent((self.nodes[n].getTimeNeeded(), "waypoint_reached", n))
            
        self.events.addEvent((50, 'measure_speed', -1))
                              
    def measure_speed(self):
        s = 0
        for node in self.nodes:
            s += node.speed
        return s / self.N
    
    def waypoint_reached(self, i):
        self.nodes[i].arrived()
        self.events.addEvent((self.nodes[i].getTimeNeeded(), "waypoint_reached", i))
        
    def simulate(self):
        # infinite loop?
        t = 0
        while not self.events.isEmpty():
            event_time, event_type, event_node = self.events.getEvent()
            t = event_time
            if event_type == 'measure_speed':
                avg_speed = self.measure_speed()
                self.events.addEvent((t + self.time_measure, 'measure_speed', -1))
                print("Avg speed = {} m/s at time {}".format(avg_speed, t))
            elif event_type == 'init':
                self.init_event_handler()
            else:
                self.waypoint_reached(event_node)

In [43]:
s = Simulation(1, 100, 100, 0, 10, 5)

In [ ]:
s.simulate()

Avg speed = 0.3567123589740575 m/s at time 50
Avg speed = 0.3567123589740575 m/s at time 55
Avg speed = 0.3567123589740575 m/s at time 60
Avg speed = 0.3567123589740575 m/s at time 65
Avg speed = 0.3567123589740575 m/s at time 70
Avg speed = 0.3567123589740575 m/s at time 75
Avg speed = 0.39104791906665426 m/s at time 80
Avg speed = 0.39104791906665426 m/s at time 85
Avg speed = 0.43944041015853763 m/s at time 90
Avg speed = 0.43944041015853763 m/s at time 95
Avg speed = 0.43944041015853763 m/s at time 100
Avg speed = 0.43944041015853763 m/s at time 105
Avg speed = 0.43944041015853763 m/s at time 110
Avg speed = 0.43944041015853763 m/s at time 115
Avg speed = 0.43944041015853763 m/s at time 120
Avg speed = 0.43944041015853763 m/s at time 125
Avg speed = 0.43944041015853763 m/s at time 130
Avg speed = 0.43944041015853763 m/s at time 135
Avg speed = 0.03615517299041748 m/s at time 140
Avg speed = 0.03615517299041748 m/s at time 145
Avg speed = 0.03615517299041748 m/s at time 150
Avg spee

Avg speed = 0.0003568816220127502 m/s at time 7135
Avg speed = 0.0003568816220127502 m/s at time 7140
Avg speed = 0.0003568816220127502 m/s at time 7145
Avg speed = 0.0003568816220127502 m/s at time 7150
Avg speed = 0.0003568816220127502 m/s at time 7155
Avg speed = 0.0003568816220127502 m/s at time 7160
Avg speed = 0.0003568816220127502 m/s at time 7165
Avg speed = 0.0003568816220127502 m/s at time 7170
Avg speed = 0.0003568816220127502 m/s at time 7175
Avg speed = 0.0003568816220127502 m/s at time 7180
Avg speed = 0.0003568816220127502 m/s at time 7185
Avg speed = 0.0003568816220127502 m/s at time 7190
Avg speed = 0.0003568816220127502 m/s at time 7195
Avg speed = 0.0003568816220127502 m/s at time 7200
Avg speed = 0.0003568816220127502 m/s at time 7205
Avg speed = 0.0003568816220127502 m/s at time 7210
Avg speed = 0.0003568816220127502 m/s at time 7215
Avg speed = 0.0003568816220127502 m/s at time 7220
Avg speed = 0.0003568816220127502 m/s at time 7225
Avg speed = 0.00035688162201275

Avg speed = 0.0003568816220127502 m/s at time 14195
Avg speed = 0.0003568816220127502 m/s at time 14200
Avg speed = 0.0003568816220127502 m/s at time 14205
Avg speed = 0.0003568816220127502 m/s at time 14210
Avg speed = 0.0003568816220127502 m/s at time 14215
Avg speed = 0.0003568816220127502 m/s at time 14220
Avg speed = 0.0003568816220127502 m/s at time 14225
Avg speed = 0.0003568816220127502 m/s at time 14230
Avg speed = 0.0003568816220127502 m/s at time 14235
Avg speed = 0.0003568816220127502 m/s at time 14240
Avg speed = 0.0003568816220127502 m/s at time 14245
Avg speed = 0.0003568816220127502 m/s at time 14250
Avg speed = 0.0003568816220127502 m/s at time 14255
Avg speed = 0.0003568816220127502 m/s at time 14260
Avg speed = 0.0003568816220127502 m/s at time 14265
Avg speed = 0.0003568816220127502 m/s at time 14270
Avg speed = 0.0003568816220127502 m/s at time 14275
Avg speed = 0.0003568816220127502 m/s at time 14280
Avg speed = 0.0003568816220127502 m/s at time 14285
Avg speed = 

Avg speed = 0.0003568816220127502 m/s at time 21695
Avg speed = 0.0003568816220127502 m/s at time 21700
Avg speed = 0.0003568816220127502 m/s at time 21705
Avg speed = 0.0003568816220127502 m/s at time 21710
Avg speed = 0.0003568816220127502 m/s at time 21715
Avg speed = 0.0003568816220127502 m/s at time 21720
Avg speed = 0.0003568816220127502 m/s at time 21725
Avg speed = 0.0003568816220127502 m/s at time 21730
Avg speed = 0.0003568816220127502 m/s at time 21735
Avg speed = 0.0003568816220127502 m/s at time 21740
Avg speed = 0.0003568816220127502 m/s at time 21745
Avg speed = 0.0003568816220127502 m/s at time 21750
Avg speed = 0.0003568816220127502 m/s at time 21755
Avg speed = 0.0003568816220127502 m/s at time 21760
Avg speed = 0.0003568816220127502 m/s at time 21765
Avg speed = 0.0003568816220127502 m/s at time 21770
Avg speed = 0.0003568816220127502 m/s at time 21775
Avg speed = 0.0003568816220127502 m/s at time 21780
Avg speed = 0.0003568816220127502 m/s at time 21785
Avg speed = 

Avg speed = 0.0003568816220127502 m/s at time 29190
Avg speed = 0.0003568816220127502 m/s at time 29195
Avg speed = 0.0003568816220127502 m/s at time 29200
Avg speed = 0.0003568816220127502 m/s at time 29205
Avg speed = 0.0003568816220127502 m/s at time 29210
Avg speed = 0.0003568816220127502 m/s at time 29215
Avg speed = 0.0003568816220127502 m/s at time 29220
Avg speed = 0.0003568816220127502 m/s at time 29225
Avg speed = 0.0003568816220127502 m/s at time 29230
Avg speed = 0.0003568816220127502 m/s at time 29235
Avg speed = 0.0003568816220127502 m/s at time 29240
Avg speed = 0.0003568816220127502 m/s at time 29245
Avg speed = 0.0003568816220127502 m/s at time 29250
Avg speed = 0.0003568816220127502 m/s at time 29255
Avg speed = 0.0003568816220127502 m/s at time 29260
Avg speed = 0.0003568816220127502 m/s at time 29265
Avg speed = 0.0003568816220127502 m/s at time 29270
Avg speed = 0.0003568816220127502 m/s at time 29275
Avg speed = 0.0003568816220127502 m/s at time 29280
Avg speed = 

...

Avg speed = 0.0003568816220127502 m/s at time 89435
Avg speed = 0.0003568816220127502 m/s at time 89440
Avg speed = 0.0003568816220127502 m/s at time 89445
Avg speed = 0.0003568816220127502 m/s at time 89450
Avg speed = 0.0003568816220127502 m/s at time 89455
Avg speed = 0.0003568816220127502 m/s at time 89460
Avg speed = 0.0003568816220127502 m/s at time 89465
Avg speed = 0.0003568816220127502 m/s at time 89470
Avg speed = 0.0003568816220127502 m/s at time 89475
Avg speed = 0.0003568816220127502 m/s at time 89480
Avg speed = 0.0003568816220127502 m/s at time 89485
Avg speed = 0.0003568816220127502 m/s at time 89490
Avg speed = 0.0003568816220127502 m/s at time 89495
Avg speed = 0.0003568816220127502 m/s at time 89500
Avg speed = 0.0003568816220127502 m/s at time 89505
Avg speed = 0.0003568816220127502 m/s at time 89510
Avg speed = 0.0003568816220127502 m/s at time 89515
Avg speed = 0.0003568816220127502 m/s at time 89520
Avg speed = 0.0003568816220127502 m/s at time 89525
Avg speed = 

Avg speed = 0.0003568816220127502 m/s at time 98565
Avg speed = 0.0003568816220127502 m/s at time 98570
Avg speed = 0.0003568816220127502 m/s at time 98575
Avg speed = 0.0003568816220127502 m/s at time 98580
Avg speed = 0.0003568816220127502 m/s at time 98585
Avg speed = 0.0003568816220127502 m/s at time 98590
Avg speed = 0.0003568816220127502 m/s at time 98595
Avg speed = 0.0003568816220127502 m/s at time 98600
Avg speed = 0.0003568816220127502 m/s at time 98605
Avg speed = 0.0003568816220127502 m/s at time 98610
Avg speed = 0.0003568816220127502 m/s at time 98615
Avg speed = 0.0003568816220127502 m/s at time 98620
Avg speed = 0.0003568816220127502 m/s at time 98625
Avg speed = 0.0003568816220127502 m/s at time 98630
Avg speed = 0.0003568816220127502 m/s at time 98635
Avg speed = 0.0003568816220127502 m/s at time 98640
Avg speed = 0.0003568816220127502 m/s at time 98645
Avg speed = 0.0003568816220127502 m/s at time 98650
Avg speed = 0.0003568816220127502 m/s at time 98655
Avg speed = 

Avg speed = 0.0003568816220127502 m/s at time 105480
Avg speed = 0.0003568816220127502 m/s at time 105485
Avg speed = 0.0003568816220127502 m/s at time 105490
Avg speed = 0.0003568816220127502 m/s at time 105495
Avg speed = 0.0003568816220127502 m/s at time 105500
Avg speed = 0.0003568816220127502 m/s at time 105505
Avg speed = 0.0003568816220127502 m/s at time 105510
Avg speed = 0.0003568816220127502 m/s at time 105515
Avg speed = 0.0003568816220127502 m/s at time 105520
Avg speed = 0.0003568816220127502 m/s at time 105525
Avg speed = 0.0003568816220127502 m/s at time 105530
Avg speed = 0.0003568816220127502 m/s at time 105535
Avg speed = 0.0003568816220127502 m/s at time 105540
Avg speed = 0.0003568816220127502 m/s at time 105545
Avg speed = 0.0003568816220127502 m/s at time 105550
Avg speed = 0.0003568816220127502 m/s at time 105555
Avg speed = 0.0003568816220127502 m/s at time 105560
Avg speed = 0.0003568816220127502 m/s at time 105565
Avg speed = 0.0003568816220127502 m/s at time 

Avg speed = 0.0003568816220127502 m/s at time 111970
Avg speed = 0.0003568816220127502 m/s at time 111975
Avg speed = 0.0003568816220127502 m/s at time 111980
Avg speed = 0.0003568816220127502 m/s at time 111985
Avg speed = 0.0003568816220127502 m/s at time 111990
Avg speed = 0.0003568816220127502 m/s at time 111995
Avg speed = 0.0003568816220127502 m/s at time 112000
Avg speed = 0.0003568816220127502 m/s at time 112005
Avg speed = 0.0003568816220127502 m/s at time 112010
Avg speed = 0.0003568816220127502 m/s at time 112015
Avg speed = 0.0003568816220127502 m/s at time 112020
Avg speed = 0.0003568816220127502 m/s at time 112025
Avg speed = 0.0003568816220127502 m/s at time 112030
Avg speed = 0.0003568816220127502 m/s at time 112035
Avg speed = 0.0003568816220127502 m/s at time 112040
Avg speed = 0.0003568816220127502 m/s at time 112045
Avg speed = 0.0003568816220127502 m/s at time 112050
Avg speed = 0.0003568816220127502 m/s at time 112055
Avg speed = 0.0003568816220127502 m/s at time 

Avg speed = 0.0003568816220127502 m/s at time 119160
Avg speed = 0.0003568816220127502 m/s at time 119165
Avg speed = 0.0003568816220127502 m/s at time 119170
Avg speed = 0.0003568816220127502 m/s at time 119175
Avg speed = 0.0003568816220127502 m/s at time 119180
Avg speed = 0.0003568816220127502 m/s at time 119185
Avg speed = 0.0003568816220127502 m/s at time 119190
Avg speed = 0.0003568816220127502 m/s at time 119195
Avg speed = 0.0003568816220127502 m/s at time 119200
Avg speed = 0.0003568816220127502 m/s at time 119205
Avg speed = 0.0003568816220127502 m/s at time 119210
Avg speed = 0.0003568816220127502 m/s at time 119215
Avg speed = 0.0003568816220127502 m/s at time 119220
Avg speed = 0.0003568816220127502 m/s at time 119225
Avg speed = 0.0003568816220127502 m/s at time 119230
Avg speed = 0.0003568816220127502 m/s at time 119235
Avg speed = 0.0003568816220127502 m/s at time 119240
Avg speed = 0.0003568816220127502 m/s at time 119245
Avg speed = 0.0003568816220127502 m/s at time 